In [1]:
import requests
import pandas as pd
import json
import itertools

In [2]:
API_KEY = "ejCxSDJxfBi4THMKcw5e"  
params = { 
        'apikey': API_KEY,
        'current_rating__decay__lt': '4',
        'order_by': '-current_rating__rating',
        'format': 'json',
        'limit': 30
         } 


In [3]:
response = requests.get('http://aligulac.com/api/v1/player/', params=params) 
data = response.json()

In [4]:
players = data['objects']
df_players = pd.DataFrame(players)

In [5]:
df_players['rating'] = df_players['current_rating'].str['rating']
df_players['vProtoss'] = df_players['current_rating'].str['tot_vp']
df_players['vTerran'] = df_players['current_rating'].str['tot_vt']
df_players['vZerg'] = df_players['current_rating'].str['tot_vz']


In [6]:
df=df_players.drop(columns=['birthday','country','romanized_name','dom_end','dom_start',
                         'current_teams','aliases','lp_name','mcnum','sc2e_id','name',
                         'total_earnings','resource_uri','tlpd_id','tlpd_db','past_teams',
                         'resource_uri','current_rating','dom_val'])

In [ ]:
# Konwersja ratingu Aligulac (skala ok. 2.5-3.0) na bardziej czytelną skalę (np. 3500-4000)
a = 1000.9197397008222
b = 997.3905539423292
df['rating'] = round((a * df['rating'] + b),0)
df['vProtoss'] = round((a * df['vProtoss'] + b),0)
df['vTerran'] = round((a * df['vTerran'] + b),0)
df['vZerg'] = round((a * df['vZerg'] + b),0)

In [8]:
df['race'] = df['race'].replace({'Z': 0,
                                 "T":1,
                                 "P":2})

C:\Users\s-a49\AppData\Local\Temp\ipykernel_18372\2766011006.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['race'] = df['race'].replace({'Z': 0,


In [9]:
ids = df['id'].tolist()
pairs = list(itertools.permutations(ids, 2))

In [10]:
base = 'http://aligulac.com/api/v1/match/'
params = {
    "apikey": API_KEY,
    "pla__in": "485,5878",
    "plb__in": "485,5878",
    "limit": 5
}


m_response = requests.get(base, params=params)
m_data = m_response.json()

In [11]:
matches = m_data['objects']
mdf = pd.DataFrame(matches)

In [ ]:
"""
import time

all_matches_df_list = []
base_url = 'http://aligulac.com/api/v1/match/'
api_params = {
    "apikey": API_KEY,
    "limit": 0 # Ustawienie limitu na 0 oznacza pobranie wszystkich rekordów
}

# Możesz zmniejszyć liczbę par na początek, żeby przetestować kod
# np. dla pierwszych 20 par: for pair in pairs[:20]:
for pair in pairs:
    player_a_id, player_b_id = pair
    
    # Przygotuj parametry zapytania dla konkretnej pary
    api_params['pla__in'] = f"{player_a_id},{player_b_id}"
    api_params['plb__in'] = f"{player_a_id},{player_b_id}"
    
    try:
        response = requests.get(base_url, params=api_params)
        response.raise_for_status() # Sprawdź, czy zapytanie się powiodło
        
        data = response.json()
        matches = data['objects']
        
        if matches: # Jeśli znaleziono jakiekolwiek mecze
            temp_df = pd.DataFrame(matches)
            all_matches_df_list.append(temp_df)
            
        print(f"Pobrano mecze dla pary: {player_a_id} vs {player_b_id}")

    except requests.exceptions.RequestException as e:
        print(f"Błąd podczas pobierania danych dla pary {pair}: {e}")

    # pauza dla api
    time.sleep(0.5)

# Połącz wszystkie ramki danych w jedną
if all_matches_df_list:
    final_matches_df = pd.concat(all_matches_df_list, ignore_index=True)
    print("Zakończono pobieranie. Stworzono finalną ramkę danych.")
    # Zapisz do pliku, żeby nie pobierać za każdym razem
    final_matches_df.to_csv("./data/raw/all_matches.csv", index=False)
else:
    print("Nie znaleziono żadnych meczów dla podanych par.")
"""

'\nimport time\n\nall_matches_df_list = []\nbase_url = \'http://aligulac.com/api/v1/match/\'\napi_params = {\n    "apikey": API_KEY,\n    "limit": 0 # Ustawienie limitu na 0 oznacza pobranie wszystkich rekordów\n}\n\n# Możesz zmniejszyć liczbę par na początek, żeby przetestować kod\n# np. dla pierwszych 20 par: for pair in pairs[:20]:\nfor pair in pairs:\n    player_a_id, player_b_id = pair\n    \n    # Przygotuj parametry zapytania dla konkretnej pary\n    api_params[\'pla__in\'] = f"{player_a_id},{player_b_id}"\n    api_params[\'plb__in\'] = f"{player_a_id},{player_b_id}"\n    \n    try:\n        response = requests.get(base_url, params=api_params)\n        response.raise_for_status() # Sprawdź, czy zapytanie się powiodło\n        \n        data = response.json()\n        matches = data[\'objects\']\n        \n        if matches: # Jeśli znaleziono jakiekolwiek mecze\n            temp_df = pd.DataFrame(matches)\n            all_matches_df_list.append(temp_df)\n            \n        p

In [13]:
all_matches = pd.read_csv("all_matches.csv")

all_matches.head

<bound method NDFrame.head of              date event                                           eventobj  \
0      2023-12-17   NaN  {'fullname': 'ESL Pro Tour 2023/24 SC2 Masters...   
1      2025-01-25   NaN  {'fullname': 'LiuLi Cup 2024 Grand Finals Main...   
2      2021-08-15   NaN  {'fullname': 'ESL Pro Tour 2021/22 DH SC2 Mast...   
3      2019-05-17   NaN  {'fullname': 'WCS 2019 Spring Main Tournament ...   
4      2022-10-11   NaN  {'fullname': 'ESL Pro Tour 2022/23 DH SC2 Mast...   
...           ...   ...                                                ...   
30555  2025-07-05   NaN  {'fullname': 'FEL 2025 Cracov Qualifier #1 LB ...   
30556  2023-12-09   NaN  {'fullname': 'SC2Online OSC Masters Cup #148 M...   
30557  2025-07-27   NaN  {'fullname': 'FEL 2025 Cracov Main Event LB Ro...   
30558  2021-01-23   NaN  {'fullname': 'Cheeseadelphia Winter Championsh...   
30559  2023-11-07   NaN  {'fullname': 'WardiTV 2023 Series Open Novembe...   

       game      id  offline  \
0